# A First Model

Suppose the data is organized in a long format where each individual appears in more than one row: 


| Id |Faa |week | sex | group |
|:--:|:--:|:---:|:---:|:-----:|
|1  |5	| 1 | G = 0 | T 
|2  |2	| 1 | B = 1 | A
|1  |8	| 3	| G = 0 | T
|1  |1	| 2 | G = 0 | T


Then, the model for the $i^{th}$ observation is:

$$ \Large
Faa_i = \alpha_0 + \alpha_{sex} \ sex_{j[i]} + \alpha_{group} \ group_{j[i]} + \alpha_{week} \ week_{j[i]} + \alpha_{j[i]}
$$

where $j[i]$ is the id of the subject corresponding to observation $i$. So for example, if we consider the third observation, then $i = 3$ and $j[i] = 1$ so that $week_{j[i]} = 3$. 

The last coefficient represents an indivual random effect. For example we can set: 

$$ \Large 
\alpha_{j} \sim \mathcal{N}\left(0, \ \sigma_j^2\right)
$$

In [1]:
library(haven)
library(tidyverse)
library(broom)
library(lme4) 

Loading tidyverse: ggplot2
Loading tidyverse: tibble
Loading tidyverse: tidyr
Loading tidyverse: readr
Loading tidyverse: purrr
Loading tidyverse: dplyr
Conflicts with tidy packages ---------------------------------------------------
filter(): dplyr, stats
lag():    dplyr, stats
Loading required package: Matrix

Attaching package: 'Matrix'

The following object is masked from 'package:tidyr':

    expand



In [2]:
aaSelect <- function(df, AA){
    aaDf <- df %>% select(id = ID, group = GROUP, age = MATERNALAGE, sex = SEXO, starts_with(AA)) %>%
                         mutate(id = factor(id), 
                                sex =  factor(sex, labels = c('girl', 'boy')),
                                group = factor(group, labels = c('teen', 'adult'))) 
    
    return(aaDf)
}

aaLong <-  function(df, AA){
    aaL <- df %>% gather(week, level, starts_with(AA))
    
    aaL[aaL == paste(AA, 'Calostrum', sep = '')] = 1
    aaL[aaL == paste(AA, 'Transition', sep = '')] = 2
    aaL[aaL == paste(AA, 'Mature2m', sep = '')] = 4
    aaL[aaL == paste(AA, 'Mature4m', sep = '')] = 16

    aaL <- aaL %>% mutate(week = as.numeric(week))
    
    return(aaL)
}

In [3]:
AA_Sex_1 <- read_sav("C:/Users/Personal/AA leche lactancia/datos/AMINOACIDOS POR SEXO_1.sav")
head(AA_Sex_1)

ID,GROUP,MATERNALAGE,SEXO,INFANTDOB,MATBMI1COLL,MATBMI215d,MATBMI32mos,MATBMI44mos,BABYWeigth1g,...,ALAGLOB,ASNGLOB,SYSGLOB,GLYGLOB,GLUGLOB,GLNGLOB,PROLINEGLOB,SERINEGLOB,TYRAMINEGLOB,new
1,0,17,1,2009-08-20,25,24,24,22,3492.7,...,21.25,0.5,1.00,10.00,87.25,23.25,2.75,8.0,1.00,0
2,0,16,2,2009-08-24,32,31,30,29,3392.9,...,18.25,0.0,0.75,13.75,100.50,40.50,2.50,9.5,0.75,0
3,0,17,2,2009-08-21,23,23,NA,NA,2993.8,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,0
4,0,16,2,2009-08-21,26,NA,NA,NA,3692.3,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,0
5,0,16,NA,2009-08-29,25,NA,NA,NA,3592.5,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,0
6,0,16,2,2009-08-27,29,28,30,31,3093.5,...,15.50,0.5,1.50,10.25,74.75,30.00,5.25,8.0,2.00,0


In [4]:
AA <- 'GLU'

AAdf <- aaSelect(AA_Sex_1, AA) %>% select(-contains('GLOB'))
head(AAdf)

id,group,age,sex,GLUCalostrum,GLUTransition,GLUMature2m,GLUMature4m
1,teen,17,girl,89,89,99,72
2,teen,16,boy,4,103,104,191
3,teen,17,boy,54,122,NA,NA
4,teen,16,boy,33,NA,NA,NA
5,teen,16,NA,23,NA,NA,NA
6,teen,16,boy,30,67,92,110


In [5]:
AAdfLong <- aaLong(AAdf, AA)
head(AAdfLong)

Warning message:
"attributes are not identical across measure variables; they will be dropped"

id,group,age,sex,week,level
1,teen,17,girl,1,89
2,teen,16,boy,1,4
3,teen,17,boy,1,54
4,teen,16,boy,1,33
5,teen,16,NA,1,23
6,teen,16,boy,1,30


In [6]:
mod1 <- lmer(level ~ sex + group + week + (1 | id), data=AAdfLong)
summary(mod1)

Linear mixed model fit by REML ['lmerMod']
Formula: level ~ sex + group + week + (1 | id)
   Data: AAdfLong

REML criterion at convergence: 1848.6

Scaled residuals: 
     Min       1Q   Median       3Q      Max 
-2.77767 -0.60556 -0.00799  0.55937  2.97839 

Random effects:
 Groups   Name        Variance Std.Dev.
 id       (Intercept)  170.5   13.06   
 Residual             1442.3   37.98   
Number of obs: 183, groups:  id, 61

Fixed effects:
            Estimate Std. Error t value
(Intercept)  62.6019     6.7297   9.302
sexboy       13.4288     6.9197   1.941
groupadult   -4.4517     6.9811  -0.638
week          3.4269     0.5003   6.849

Correlation of Fixed Effects:
           (Intr) sexboy grpdlt
sexboy     -0.695              
groupadult -0.539  0.239       
week       -0.334  0.009 -0.066

In [7]:
fixef(mod1)

(Intercept)      sexboy  groupadult        week 
  62.601865   13.428827   -4.451673    3.426850

In [8]:
ranef(mod1)

$id
   (Intercept)
1    1.5871717
2    1.5297598
3    1.3058349
4   -4.9113302
6   -6.7371847
7   -0.5570417
8   18.3044336
9   -3.4882281
10   5.9906794
11   1.1168058
12  -5.1831456
13   6.1046708
14  -4.9140177
15  -1.5201809
16  14.5663848
17  -4.1084707
18   7.4978647
19  -3.5848766
20 -13.2383741
21  -4.1259803
22  -3.0921824
23  -2.4030585
24  -6.6027946
25   2.1716773
26  10.5771636
28   0.3029861
29  -4.1707363
30   6.8270252
31  -8.7437247
32   3.4331884
33   5.1770743
34  -2.2230828
35   2.2640265
36  -9.9409355
37   0.5437709
38   2.1663165
39  -1.9214889
41 -10.6855183
42   0.5282556
45  -2.8549668
46 -10.3462830
47  -5.9207462
48  -7.6660470
49  -4.7125302
50  -2.1149511
51  -4.7612795
52   1.0326749
53   1.6519179
54   1.3308716
55   9.3798810
56  -0.4348836
57  14.4135119
58   4.5413354
59   0.5475912
61  15.2161279
62   5.3668011
63  -0.7559300
64  -1.4554346
65  -3.7610815
66   1.9459447
67  -0.4852612
